In [1]:
# boilerplate
import numpy as np
import tensorflow as tf

sess = tf.Session()
import tensorflow_fold as td

# Define Hierachical LSTM NNet

In [2]:
def chr_enum(c):
    """map all common characters to [1...94] any other to 0"""
    enum = ord(c) - ord(' ')
    return enum if 0 < enum <= 94 else 0

def GetLastState():
    """Composition of blocks that gets last state vector from LSTM output"""
    return td.GetItem(1) >> td.GetItem(1)

def CascadingRNN(cell, rnn_name=None):
    """Returns a concatenation of input and output state sequence of RNN(cell)
    """
    return td.AllOf(
        td.Identity(),
        td.RNN(cell, name=rnn_name) >> td.GetItem(0),
    ) >> td.Zip() >> td.Map(td.Concat())

char_cell = td.ScopedLayer(tf.contrib.rnn.BasicLSTMCell(num_units=64), 'char_cell')
word_cell_1 = td.ScopedLayer(tf.contrib.rnn.BasicLSTMCell(num_units=128), 'word_cell_1')
word_cell_2 = td.ScopedLayer(tf.contrib.rnn.BasicLSTMCell(num_units=128), 'word_cell_2')

word_vector = (
    td.InputTransform(lambda word: [chr_enum(c) for c in word]) >>
    td.Map(
        td.Scalar('int32') >>
        td.Function(td.Embedding(95, 8, mod_inputs=False))
    ) >>
    td.RNN(char_cell) >> GetLastState()
)

sentence_vector = (
    td.InputTransform(lambda text: text.split(' ')) >>
    td.Map(word_vector) >>
    CascadingRNN(word_cell_1) >> td.RNN(word_cell_2) >> GetLastState()
)

text_to_logits = sentence_vector >> td.FC(1, activation=None)

target = td.Vector(1)

compiler = td.Compiler.create((text_to_logits, target))
[logits, labels] = compiler.output_tensors
loss = tf.losses.sigmoid_cross_entropy(labels, logits)
train_op = tf.train.AdamOptimizer().minimize(loss)
sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()
saver.restore(sess, 'resources/IMDBHierarchicalLSTMBiggerParams/model.ckpt')

def sigmoid(x):
    return 1/(1 + np.exp(-x))

def get_sentiment(text):
    return sigmoid(text_to_logits.eval(text, session=sess))

/home/andershuss/.virtualenvs/tensorflow-fold/local/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


# Test some sentences

In [3]:
get_sentiment('I would never watch it again')

array([ 0.2936632], dtype=float32)

In [5]:
queries = [
    'it was ok',
    'it was great',
    'it was a disappointment',
    'it was a great disappointment',
    'I would never watch it again',
    'I would watch it again',
    'it was booring',
    'this doesn\'t really mean anything',
    'this does really mean something'
]

In [6]:
for query in queries:
    print('{}: {}'.format(query, get_sentiment(query)))

it was ok: [ 0.84210473]
it was great: [ 0.99832338]
it was a disappointment: [ 0.01001158]
it was a great disappointment: [ 0.13792171]
I would never watch it again: [ 0.2936632]
I would watch it again: [ 0.79767078]
it was booring: [ 0.07158819]
this doesn't really mean anything: [ 0.02003623]
this does really mean something: [ 0.80717897]


# Evaluate on test data

In [45]:
from tffold_additions.imdb.dataloader import IMDBDataLoader
test_dl = IMDBDataLoader('/home/andershuss/Data/aclImdb/test/')

In [52]:
epoch = train_dl.get_epoch_iterator(batch_size=16)
test_feed_dicts = [compiler.build_feed_dict(next(epoch)) for _ in range(20)]

In [53]:
probas_pred = tf.nn.sigmoid(logits)
labels_pred = tf.cast(probas_pred > 0.5, 'float32')
matches = tf.cast(tf.equal(labels_pred, labels), 'float32')
n_matches = tf.reduce_sum(matches)
accuracy = tf.reduce_mean(matches)

In [54]:
test_score = np.mean([sess.run(accuracy, fd) for fd in test_feed_dicts])

In [55]:
test_score

0.89687502